In [ ]:
import pandas as pd

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)
pd.plotting.register_matplotlib_converters()

import numpy as np
import seaborn as sns
import missingno as mgn

import plotly.express as px

In [ ]:
df = pd.read_csv('../data/full_data.csv')
df

In [ ]:
page106523 = df.query('page_id == 106523')

1. Concat the authors by `page_id`, `day`, `publish_date`, `word_count`

In [ ]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_row', 139)

In [ ]:
page106523_aggr = page106523[['page_id', 'date', 'publish_date', 'word_count', 'authors']]
page106523_aggr.loc[:, 'authors'] = page106523_aggr.authors.apply(lambda s: s.replace(' und ', ';'))

page106523_aggr = page106523_aggr[['page_id', 'date', 'publish_date', 'word_count', 'authors']]\
    .groupby(['page_id', 'date', 'publish_date', 'word_count']) \
        .agg(lambda auth: ';'.join(auth))

page106523_aggr = page106523_aggr.authors.apply(lambda auths: sorted(list(set(auths.split(';'))))).to_frame()

It worked!

In [ ]:
df_aggr = df[['page_id', 'date', 'publish_date', 'word_count', 'authors']]
df_aggr.loc[:, 'authors'] = df_aggr.authors.apply(lambda s: s.replace(' und ', ';'))

df_aggr = df_aggr[['page_id', 'date', 'publish_date', 'word_count', 'authors']]\
    .groupby(['page_id', 'date', 'publish_date', 'word_count']) \
        .agg(lambda auth: ';'.join(auth))

df_aggr = df_aggr.authors.apply(lambda auths: sorted(list(set(auths.split(';'))))).to_frame()

In [ ]:
df = pd.merge(df.drop(['authors'], axis=1), 
        df_aggr,
        on=['page_id', 'date', 'publish_date', 'word_count'], how='left')

In [ ]:
page1014826 = df.query('page_id == 1014826')

Now recalculate the versions

In [ ]:
df.loc[:, 'authors'] = df.loc[:, 'authors'].apply(lambda l: str(l))

In [ ]:
#df.loc[:, 'authors'] = df.authors.str.removeprefix("['")
#df.loc[:, 'authors'] = df.authors.str.removesuffix("']")

### This should do the same thing!
df.authors.apply(''.join)

In [ ]:
temp = df[['page_id', 'word_count', 'publish_date', 'authors']].copy()
temp.drop_duplicates(inplace=True)

In [ ]:
temp = temp.drop_duplicates()
temp = temp.sort_values('publish_date')

wc_versions = temp.groupby('page_id')['word_count'].transform(lambda x: pd.factorize(x)[0])
publish_versions = temp.groupby('page_id')['publish_date'].transform(lambda x: pd.factorize(x)[0])
authors_versions = temp.groupby('page_id')['authors'].transform(lambda x: pd.factorize(x)[0])

version_count = 10000*wc_versions + 1000*publish_versions + 1*authors_versions
temp['ver_id_wc'] = wc_versions
temp['ver_id_pub'] = publish_versions
temp['ver_id_auth'] = authors_versions
temp['version_id_raw'] = version_count
temp['version_id_new'] = temp.groupby('page_id')['version_id_raw'].transform(lambda x: pd.factorize(x)[0])

df_versions = pd.merge(left=df, right=temp.drop(['ver_id_wc', 'ver_id_pub', 'ver_id_auth', 'version_id_raw'], axis=1),
         on=['page_id', 'word_count', 'publish_date', 'authors'],
         how='left')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)

In [ ]:
page106523 = df_versions.query('page_id == 106523')

In [ ]:
df_new = pd.read_csv('../data/aggr_data.csv')

In [ ]:
df_versions.head()

In [ ]:
df_new.head()

## 2. 